In [1]:
import numpy as np
import cv2 as cv
import struct


In [2]:
def imshow(frame) :
   # frame=cv.resize(frame,(600,480))
    cv.imshow('Figure', frame)
    cv.waitKey(0)
    #video.release()
    cv.destroyAllWindows()


# 3D frame segmentor
def frame_segmenter (frame,segment_size):
    height, width, channels = frame.shape
    
    rows = height // segment_size
    columns = width // segment_size

    # Initialize an empty array to store the segments
    segments = np.empty((rows, columns, segment_size, segment_size, channels), dtype=frame.dtype)

    # Split the image into segments
    for i in range(rows):
        for j in range(columns):
            segment = frame[i*segment_size:(i+1)*segment_size, j*segment_size:(j+1)*segment_size, :]
            segments[i, j] = segment
    return segments


#========================================================================================================================
# 2D frame segmentor
def segmentor(original_list,segment_size):

    shp = original_list.shape
    M=shp[0]
    N=shp[1]
    
    # Calculate the number of segments in rows and columns
    num_segments_m = M // segment_size
    num_segments_n = N // segment_size

    # Create an empty array to store the segmented data
    segmented_list = np.empty((num_segments_m, num_segments_n, segment_size, segment_size))

    for i in range(num_segments_m):
        for j in range(num_segments_n):
            segment = original_list[i*segment_size:(i+1)*segment_size, j*segment_size:(j+1)*segment_size]
            segmented_list[i, j] = segment

    print(segmented_list.shape)
    return segmented_list
#=========================================================================================================================

def cell2mat(segmentedFrame):
    tmp_rslt=[]
    resultFrame=[]
    shape= segmentedFrame.shape
    rows=shape[0]
    columns=shape[1]

    for ii in range(rows):
        for jj in range(columns):
            tmp_rslt=cat((tmp_rslt,segmentedFrame[ii, jj]),axis=1)

        resultFrame=cat((resultFrame,tmp_rslt),axis=0)
        tmp_rslt=[]
    
    return resultFrame
#========================================================================================================================

def cat(tupleOfArrays, axis=0):
    # deals with problems of concating empty arrays
    # also gives better error massages

    # first check that the input is correct
    assert isinstance(tupleOfArrays, tuple), 'first var should be tuple of arrays'

    firstFlag = True
    res = np.array([])

    # run over each element in tuple
    for i in range(len(tupleOfArrays)):
        x = tupleOfArrays[i]
        if len(x) > 0:  # if an empty array\list - skip
            if isinstance(x, list):  # all should be ndarray
                x = np.array(x)
            if x.ndim == 1:  # easier to concat 2d arrays
                x = x.reshape((1, -1))
            if firstFlag:  # for the first non empty array, just swich the empty res array with it
                res = x
                firstFlag = False
            else:  # actual concatination

                # first check that concat dims are good
                if axis == 0:
                    assert res.shape[1] == x.shape[1], "Error concating vertically element index " + str(i) + \
                                                       " with prior elements: given mat shapes are " + \
                                                       str(res.shape) + " & " + str(x.shape)
                else:  # axis == 1:
                    assert res.shape[0] == x.shape[0], "Error concating horizontally element index " + str(i) + \
                                                       " with prior elements: given mat shapes are " + \
                                                       str(res.shape) + " & " + str(x.shape)

                res = np.concatenate((res, x), axis=axis)
    return res

#====================================================================================================================
def eval_matric(reference_frame,decoded_frame):
    import cv2
    from skimage.metrics import peak_signal_noise_ratio, structural_similarity

    # # Load two frames
    # frame1_reference = cv2.imread('frame1.jpg')
    # frame2_decoded = cv2.imread('frame2.jpg')

    # Convert frames to grayscale (assuming frames are color images)
    gray_frame1_reference = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2GRAY)
    gray_frame2_decoded = cv2.cvtColor(decoded_frame, cv2.COLOR_BGR2GRAY)

    # Calculate PSNR
    psnr_value = peak_signal_noise_ratio(gray_frame1_reference, gray_frame2_decoded)

    # Calculate SSIM
    ssim_value, _ = structural_similarity(gray_frame1_reference, gray_frame2_decoded, full=True)

#     Print results
#     print(f'PSNR: {psnr_value:.2f} dB')
#     print(f'SSIM: {ssim_value:.4f}')
    return psnr_value, ssim_value

In [ ]:

# # Open the video file
video = cv.VideoCapture('s1.avi')
# # Read the first frame
ret, frame_C = video.read()

segment_size=16
height, width, channels = frame_C.shape
rows = height // segment_size #rows=160
columns = width // segment_size
frame_C=frame_C[0:rows*segment_size,0:columns*segment_size,:]
frame_C_segmented=frame_segmenter(frame_C,segment_size)

#Creating object for storing the encoded video
v_enc = cv.VideoWriter('enc_s1_1.mp4', cv.VideoWriter_fourcc(*'x264'), video.get(cv.CAP_PROP_FPS), (columns*segment_size, rows*segment_size))
v_enc.set(cv.VIDEOWRITER_PROP_QUALITY, 100)

p=v_enc.write(frame_C)  # storing 1st frame as reference frame in the encoded video
missmatch_threshold=2 # Missmatch segments threshold value in %
pct=50 #pixel correction threshold
f_count=200
f_countt=0
mmsc=0  #missmatched segment counter
miss_match_segments=[]

encoded_segments=[]
evf=1  # encoded video frame counter

# Empty segmented frame for storing non-redundant segments
encoded_frame_segmented = np.empty((rows, columns, segment_size, segment_size, channels)) 
tmp_encoded_frame = encoded_frame_segmented.copy()
frame_C=frame_C[0:rows*segment_size,0:columns*segment_size,:]
frame_C_segmented=frame_segmenter(frame_C,segment_size)



with open("data_s1.bin", "wb") as file:  # The .bin file is used to store the index location of the non-redundant segments and
    #                                       will be used in the reconstruction of decoded frames. The decoded frames will be
    #                                         used in calculating PSNR and SSIM of the decoded video obtained from proposed algorithm

    # while (video.isOpened()): #will run till the end of file
    while (f_count): #will run till the value of f_count become 0
        f_count-=1
        f_countt+=1  
        print('frames processed=',f_countt )

#         reading Next frame frame_N
        ret, frame_N = video.read()

        


        # Check if the frame was successfully read
        if ret==True:
            frame_N=frame_N[0:rows*segment_size,0:columns*segment_size,:]
            frame_N_segmented=[]
            frame_N_segmented=frame_segmenter(frame_N,segment_size)
            red_status=1   # redundancy status of the current frame segment (frame_N)
  
            for ii in range(rows):
            
                for jj in range(columns):
                    segment_diff=np.abs(np.int16(frame_N_segmented[ii,jj])-np.int16(frame_C_segmented[ii,jj])) # calculating the difference segment
                    segment_diff[segment_diff<=pct]=0
                    miss_match_count=np.count_nonzero(segment_diff) # to count the non zero elements in difference segment
                    miss_match=miss_match_count/np.size(segment_diff)*100 # calculating the missmatch in percentage


                    if (miss_match>=missmatch_threshold):
                        ir,jc=divmod(mmsc,columns)
                        mmsc+=1 # mismatched segments counter
                        mm_segment=frame_N_segmented[ii,jj].copy()
                        red_status=0
                        encoded_frame_segmented[ir,jc]=mm_segment
                
                        num = [ii, jj]
                        binary_data = struct.pack("BB", *num)
                        file.write(binary_data)
    
                        if mmsc==(rows*columns):
                            
                            de_f=np.uint8(cell2mat(encoded_frame_segmented))
                            v_enc.write(de_f)
                            cv.imwrite('encode.jpg',de_f)
                            encoded_frame_segmented=tmp_encoded_frame.copy()  
                            mmsc=0
                            evf+=1       # encoded video frames counter
                            print('encoded frame filled' )
                                    
                            
            if red_status==1:   # redundent frame found
                num = [255, 0]
                binary_data = struct.pack("BB", *num)
                file.write(binary_data)
                mmsc+=1
                
            else:       # frame processing completed
                num = [255, 255]
                binary_data = struct.pack("BB", *num)
                file.write(binary_data)
  
                mmsc+=1

        else:
            break
    print("Process completed")
    de_f=np.uint8(cell2mat(encoded_frame_segmented))
    v_enc.write(de_f)
    evf+=1
    video.release()
    v_enc.release()
    print('total frames in encoded video =',evf)
    cv.destroyAllWindows()



In [3]:
#Encoded frame/video decoder and SSIM, PSNR evaluation
# The encode video is firstly reconstructed as the encoded files only contains the non redundant segments and then its PSNR and SSIM value
#  is calculated.

dec_frame_count=0
dec_seg_count=0
psnr_val=[]
ssim_val=[]

# Opening the reference and encoded video
enc_video = cv.VideoCapture('enc_s1.mp4') # Proposed algorithm encoded video

ref_video=cv.VideoCapture('s1.avi')  # Original video

#  Read the first frame
ret, frame_ref = enc_video.read()

ret1, reference_frame = ref_video.read()

segment_size=16
height, width, channels = frame_ref.shape
rows = height // segment_size #rows=160
columns = width // segment_size



reference_frame=reference_frame[0:rows*segment_size,0:columns*segment_size,:]

psnr_value,ssim_value=eval_matric(reference_frame,frame_ref)  # eval_matric(reference_frame,decoded_frame)


psnr_val.append(psnr_value)
ssim_val.append(ssim_value)
print(f'PSNR: {psnr_value:.2f} dB')
print(f'SSIM: {ssim_value:.4f}')

if ret==True:

   
    
    frame_ref_segmented=frame_segmenter(frame_ref,segment_size)
    tmp_ref_segmented=frame_ref_segmented.copy()
#     imshow(frame_ref)
    
    print('reference frame successfuly extracted')
else:
    print('reference frame extraction failed')
    
with open("data_s1.bin", "rb") as file:
    
    while (enc_video.isOpened()): #will run till the end of file
        ret, frame_enc = enc_video.read()

        ret1, reference_frame = ref_video.read()
        reference_frame=reference_frame[0:rows*segment_size,0:columns*segment_size,:]
        
        if ret==True:

            print('encoded frame successfuly extracted')
            frame_enc_segmented=frame_segmenter(frame_enc,segment_size)
            
            for ii_dec in range(rows):

                for jj_dec in range(columns):
                    
                    dec_seg_count+=1  # decoded segments count                   
                    binary_data = file.read(2)  
                    
                    if binary_data:

                        i_d,j_d = struct.unpack("BB", binary_data)

                        if (i_d==255) & (j_d==255):
                            dec_frame_count+=1  # decoded frames count
                            dec_frame=np.uint8(cell2mat(frame_ref_segmented))
                            copy_dec_frame=dec_frame.copy()  #============================= not used
                            print('frame successfully decoded',dec_frame_count)

                            psnr_value,ssim_value=eval_matric(reference_frame,frame_ref)  # eval_matric(reference_frame,decoded_frame)
                            print(f'PSNR: {psnr_value:.2f} dB')
                            print(f'SSIM: {ssim_value:.4f}')
                            psnr_val.append(psnr_value)
                            ssim_val.append(ssim_value)

                            frame_ref_segmented=tmp_ref_segmented.copy()                    

                        elif (i_d==255) & (j_d==0):
                            dec_frame_count+=1  # decoded frames count
                            print('frame successfully decoded',dec_frame_count)
                            psnr_value,ssim_value=eval_matric(reference_frame,frame_ref)  # eval_matric(reference_frame,decoded_frame)
                            print(f'PSNR: {psnr_value:.2f} dB')
                            print(f'SSIM: {ssim_value:.4f}')
                            psnr_val.append(psnr_value)
                            ssim_val.append(ssim_value)
                        
                        else:
                            dec_seg=frame_enc_segmented[ii_dec,jj_dec] #---------------

                            frame_ref_segmented[i_d,j_d]=dec_seg.copy()  #==========================


            dec_frame=np.uint8(cell2mat(frame_ref_segmented))
            # imshow(dec_frame)
        else:
            print('decoding process completed')
            print('Proposed algorithm')
            print(f'SSIN ={np.mean(ssim_val):.4f}','\n',f'PSNR ={np.mean(psnr_val):.2f} dB')
            break
            
enc_video.release()
ref_video.release()
cv.destroyAllWindows()

PSNR: 44.07 dB
SSIM: 0.9978
reference frame successfuly extracted
encoded frame successfuly extracted
frame successfully decoded 1
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 2
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 3
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 4
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 5
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 6
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 7
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 8
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 9
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 10
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 11
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 12
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 13
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 14
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 15
PSNR: 44.07 dB
SSIM: 0.9978
frame successfully decoded 16
PSNR: 4